In [1]:
!pip install torchinfo

In [2]:
import time
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import numpy as np
import cv2
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
from torchinfo import summary
import torchvision.models as models
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
import PIL
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import OrderedDict
import platform
import psutil
import random
import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from torchvision.transforms import ColorJitter, RandomRotation, RandomResizedCrop
from torchvision.transforms.functional import gaussian_blur
from PIL import ImageOps
from tabulate import tabulate

In [3]:
cpu_info = platform.processor()

ram_info = psutil.virtual_memory()
total_ram_gb = ram_info.total / (1024 ** 3)

try:
    gpu_info = !nvidia-smi --query-gpu=gpu_name --format=csv
    gpu_name = gpu_info[1]
except:
    gpu_name = "No GPU available"

print("CPU:", cpu_info)
print("Total RAM (GB):", round(total_ram_gb, 2))
print("GPU:", gpu_name)

CPU: x86_64
Total RAM (GB): 31.35
GPU: Tesla P100-PCIE-16GB


In [4]:
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [5]:
Dataset = "/kaggle/input/melanoma-cancer-dataset"

file_paths = []
labels = []

for class_name in os.listdir(Dataset):
    class_dir = os.path.join(Dataset, class_name)
    for image_name in os.listdir(class_dir):
        file_paths.append(os.path.join(class_dir, image_name))
        labels.append(class_name)

df = pd.DataFrame({"file_path": file_paths, "label": labels})
df = df.sample(frac=1).reset_index(drop=True)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision.models as models
import torchvision.transforms as T
import torchvision.transforms.functional as F
import torchvision.utils as vutils

from torchvision.datasets import CIFAR10

from torch.utils.data import DataLoader, Dataset

import numpy as np
import os
from tqdm import tqdm
from PIL import Image

In [7]:
class ContrastiveDataset(Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        img, _ = self.dataset[index]
        img1 = self.transform(img)
        img2 = self.transform(img)
        return img1, img2

    def __len__(self):
        return len(self.dataset)

In [8]:
transform = T.Compose([
    T.RandomResizedCrop(size=32),
    T.RandomHorizontalFlip(),
    T.ColorJitter(0.5, 0.5, 0.5, 0.5),
    T.RandomGrayscale(p=0.2),
    T.ToTensor()
])

In [9]:
train_dataset = CIFAR10(root='./data', download=True, train=True)
contrastive_dataset = ContrastiveDataset(train_dataset, transform)
train_loader = DataLoader(contrastive_dataset, batch_size=256, shuffle=True)

100%|██████████| 170M/170M [00:11<00:00, 14.8MB/s] 


In [10]:
class SimCLR(nn.Module):
    def __init__(self, base_model, projection_dim=128):
        super(SimCLR, self).__init__()
        self.encoder = base_model
        self.encoder.fc = nn.Identity()  # remove original fc
        self.projection_head = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, projection_dim)
        )

    def forward(self, x):
        features = self.encoder(x)
        projections = self.projection_head(features)
        return projections

In [11]:
model = SimCLR(models.resnext101_32x8d(pretrained=False))


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [12]:
def nt_xent_loss(z_i, z_j, temperature=0.5):
    batch_size = z_i.size(0)
    z = torch.cat([z_i, z_j], dim=0)
    z = F.normalize(z, dim=1)

    similarity = torch.matmul(z, z.T) / temperature
    # create labels for each sample
    labels = torch.arange(batch_size).to(z.device)
    labels = torch.cat([labels, labels], dim=0)

    # mask out the diagonal elements (similarity with itself)
    mask = torch.eye(batch_size * 2, dtype=torch.bool).to(z.device)
    similarity = similarity.masked_fill(mask, -9e15)

    # get the similarity between positive pairs
    positives = torch.diag(similarity, batch_size) + torch.diag(similarity, -batch_size)
    numerator = torch.exp(positives)

    # calculate the denominator by summing the exponentiated similarity scores for all samples
    # except the positive pair for each sample
    # mask out the positive pairs
    mask_positives = torch.eye(batch_size * 2, dtype=torch.bool).to(z.device)
    for i in range(batch_size):
        mask_positives[i, batch_size + i] = True
        mask_positives[batch_size + i, i] = True
    # get the denominator by summing the exponentiated similarity scores for all samples
    # except the positive pair for each sample
    denominator = torch.exp(similarity).masked_fill(mask_positives, 0).sum(dim=1)
     # Fix: Repeat numerator to match denominator size
    numerator = numerator.repeat(2)  # Now numerator shape is (512,)
    # remove the zeros in denominator
    denominator = denominator + numerator
    # calculate the loss
    loss = -torch.log(numerator / denominator).mean()

    return loss

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

Device: cuda


In [ ]:
import os
import torch
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

# Assume model, train_loader, nt_xent_loss, device are defined properly

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)
save_dir = '/kaggle/working'  # Save locally in Kaggle environment

pretext_losses = []

for epoch in range(100):  # 100 epochs
    model.train()
    total_loss = 0
    for img1, img2 in tqdm(train_loader):
        img1, img2 = img1.to(device), img2.to(device)
        z1 = model(img1)
        z2 = model(img2)
        
        # Normalize embeddings before loss
        z = torch.cat([z1, z2], dim=0)
        z = F.normalize(z, p=2, dim=1)
        
        # Compute NT-Xent loss (make sure your loss uses normalized z1, z2 or z)
        loss = nt_xent_loss(z1, z2)  # Or update inside nt_xent_loss to normalize if needed
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    pretext_losses.append(avg_loss)
    print(f"Epoch [{epoch+1}/10] Pretext Loss: {avg_loss:.4f}")
    
    # Save model checkpoint after each epoch
    model_path = os.path.join(save_dir, 'simclr_pretask_resnext101_32x8d.pth')
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")



100%|██████████| 196/196 [05:56<00:00,  1.82s/it]


Epoch [1/10] Pretext Loss: 4.2497
Model saved to /kaggle/working/simclr_pretask_resnext101_32x8d.pth


100%|██████████| 196/196 [05:56<00:00,  1.82s/it]


Epoch [2/10] Pretext Loss: 4.2431
Model saved to /kaggle/working/simclr_pretask_resnext101_32x8d.pth


100%|██████████| 196/196 [05:55<00:00,  1.82s/it]


...

100%|██████████| 196/196 [05:55<00:00,  1.82s/it]


Epoch [23/10] Pretext Loss: 4.2430
Model saved to /kaggle/working/simclr_pretask_resnext101_32x8d.pth


100%|██████████| 196/196 [05:55<00:00,  1.81s/it]


Epoch [24/10] Pretext Loss: 4.2430
Model saved to /kaggle/working/simclr_pretask_resnext101_32x8d.pth


 26%|██▌       | 50/196 [01:30<04:24,  1.82s/it]

In [ ]:
save_dir = '/kaggle/working'
model_path = os.path.join(save_dir, 'simclr_classification.pth')

torch.save(model.state_dict(), model_path)

model.load_state_dict(torch.load(model_path, map_location=device))

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
import torchvision.transforms as T
from tqdm import tqdm
import os

# Assume 'model' is your pretrained SimCLR model with a resnext101_32x8d encoder
# and 'device' is your CUDA or CPU device

# Freeze encoder weights
for param in model.encoder.parameters():
    param.requires_grad = False

# Fix classifier input size (2048 for resnext101_32x8d encoder output) and output classes=10
classifier = nn.Linear(2048, 10).to(device)

optimizer = optim.Adam(classifier.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

transform = T.Compose([T.ToTensor()])
labeled_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
labeled_loader = DataLoader(labeled_dataset, batch_size=256, shuffle=True)

downstream_losses = []
downstream_accuracies = []

save_dir = '/content/drive/MyDrive/SelfSupervise'  # Your save directory
os.makedirs(save_dir, exist_ok=True)

for epoch in range(15):
    classifier.train()
    total_loss = 0
    correct, total = 0, 0

    for imgs, labels in tqdm(labeled_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        with torch.no_grad():
            features = model.encoder(imgs)

        logits = classifier(features)
        loss = criterion(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    avg_loss = total_loss / len(labeled_loader)
    accuracy = (correct / total) * 100
    downstream_losses.append(avg_loss)
    downstream_accuracies.append(accuracy)

    print(f"Epoch [{epoch+1}/15] Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

    save_path = os.path.join(save_dir, 'simclr_classifier.pth')
    torch.save(classifier.state_dict(), save_path)
    print(f"Classifier weights saved to {save_path}")



In [ ]:
import matplotlib.pyplot as plt

# Plot SimCLR pretext loss
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(pretext_losses) + 1), pretext_losses, marker='o', label='Contrastive Loss')
plt.title('SimCLR Pretext Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('NT-Xent Loss')
plt.legend()
plt.grid()
plt.show()

# Plot downstream classifier loss
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(downstream_losses) + 1), downstream_losses, marker='o', color='red', label='Classifier Loss')
plt.title('Downstream Classifier Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Cross-Entropy Loss')
plt.legend()
plt.grid()
plt.show()

# Plot downstream classifier accuracy
plt.figure(figsize=(8, 5))
plt.plot(range(1, len(downstream_accuracies) + 1), downstream_accuracies, marker='s', color='green', label='Classifier Accuracy')
plt.title('Downstream Classifier Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as T

# Prepare test dataset and dataloader
transform = T.ToTensor()
test_dataset = CIFAR10(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

all_preds, all_labels = [], []

classifier.eval()
model.encoder.eval()

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        features = model.encoder(imgs)
        logits = classifier(features)
        preds = logits.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=test_dataset.classes))
print(f"Test Accuracy: {accuracy_score(all_labels, all_preds) * 100:.2f}%")
